# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Цели проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
import matplotlib.pyplot as plt
from joblib import dump
from sklearn.dummy import DummyClassifier
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В файле представлены:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [2]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Пропусков нет, можно идти дальше

In [3]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

В выборке, значений ультра больше, чем смарт. Можно сделать вывод, что этот датасет не сбалансирован

In [4]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Наблюдается слишком высокое значение корреляции между calls и minutes, поэтому удалим столбец minutes.

In [5]:
df = df.drop('minutes', axis=1)
df.head()

,calls,messages,mb_used,is_ultra
0,40.0,83.0,19915.42,0
1,85.0,56.0,22696.96,0
2,77.0,86.0,21060.45,0
3,106.0,81.0,8437.39,1
4,66.0,1.0,14502.75,0


Вывод: открыл и изучил файл, проверил пропуски, кол-во предложенных тарифов, удалил столбец minutes

## Разбейте данные на выборки

In [7]:
# Отделим тестовую и валидационную выборки
df_train_valid, df_test = train_test_split(df, test_size=0.15, random_state=12345)

In [8]:
df_train, df_valid = train_test_split(df_train_valid, test_size=0.15, random_state=12345)

In [9]:
# сохраняем признаки и предскзания
features_test = df_test.drop('is_ultra', axis=1) 
target_test = df_test['is_ultra']

features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

In [10]:
# значения в процентах
print(f'Тестовая выборка составляет: {df_test.shape[0]/df.shape[0]*100} % от исходного датасета df')
print(f'Валидационная выборка составляет: {df_valid.shape[0]/df.shape[0]*100} % от исходного датасета df')
print(f'Обучающая выборка составляет: {df_train.shape[0]/df.shape[0]*100} % от исходного датасета')

Тестовая выборка составляет: 15.028002489110143 % от исходного датасета df
Валидационная выборка составляет: 12.756689483509644 % от исходного датасета df
Обучающая выборка составляет: 72.2153080273802 % от исходного датасета


Вывод:
- разбил данные на выборки 3:1:1
- импортировал необходимые библиотеки

## Исследуйте модели

Решающее дерево

In [11]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion='gini') #Обучение с заданной глубиной дерева
    model_dt.fit(features_train, target_train) #Обучение модели
    result_valid = model_dt.score(features_valid, target_valid) #Качество модели
    if result_valid > best_result:
        best_model = model_dt
        best_result = result_valid
        best_depth = depth
print('Accuracy лучшей модели:', best_result)
print('Глубина лучшей модели:', best_depth)

Accuracy лучшей модели: 0.8219512195121951
Глубина лучшей модели: 6


Случайный лес

In [12]:
best_model = None
best_result = 0
for est in range(1, 11):
    for depth in range (1, 11):
        model_rf= RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, criterion='gini') # обучение модели с заданным количеством деревьев
        model_rf.fit(features_train, target_train) # обучение модели на тренирововчной выборке
        result_valid = model_rf.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
        if result_valid > best_result:
            best_model = model_rf # сохраните наилучшую модель
            best_result = result_valid #  сохраните наилучшее значение метрики accuracy на валидационных данных
print('Accuracy лучшей модели:', best_result)
print('Глубина лучшей модели:', best_depth)

Accuracy лучшей модели: 0.824390243902439
Глубина лучшей модели: 6


In [13]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_lr.fit(features_train, target_train)
result = model_lr.score(features_valid, target_valid)
print('Accuracy логистической регрессии:', result)

Accuracy логистической регрессии: 0.6853658536585366


Таким образом, лучший показатель у модели Случайного дерева.

## Проверьте модель на тестовой выборке

Проверим модель на тестовой выборке

In [14]:
predictions_test_dt = model_dt.predict(features_test) #предсказания на тестовой выборке
accuracy_dt = accuracy_score(target_test, predictions_test_dt) #считаем долю ответов
print('Доля правильных ответов accuracy в модели решающее дерево:', accuracy_dt)

Доля правильных ответов accuracy в модели решающее дерево: 0.7681159420289855


Навсякий случай, проверим модель случайного леса

In [15]:
predictions_test_rf = model_rf.predict(features_test) #предсказания на тестовой выборке
accuracy_rf = accuracy_score(target_test, predictions_test_rf) #считаем долю ответов
print('Доля правильных ответов accuracy в модели решающее дерево:', accuracy_rf)

Доля правильных ответов accuracy в модели решающее дерево: 0.782608695652174


Вывод: на валидационной выборке, модель решающего дерева и случайного леса почти одинаковые. Но на тестовой выборке доля правильных ответов у случаного леса выше. И та и другая модель показали значения больше 0.72
Валидационная выборка необходима, для того чтобы проверить переобучилась ли модель или наоборот. Тестовая выборка нужна для контрольной проверки модели. Лучшая модель определется на этапе исследования. В зависимости от количества выборок.

## (бонус) Проверьте модели на адекватность

Благодоря интернету, можно попробовать проверить модель на адекватность

In [16]:
model_dummy = DummyClassifier(strategy='most_frequent', random_state=12345) # создадим модель model_dummy
model_dummy.fit(features_train, target_train) # обучим модель model_dummy на обучающей выборке
model_dummy.predict(features_test)
accuracy_dummy = model_dummy.score(features_test, target_test) # посчитаем качество модели на тестовой выборке
print(f'Наивный прогноз самого частого класса ("0"): {accuracy_dummy:.2%}')

Наивный прогноз самого частого класса ("0"): 69.77%


Наивный прогноз самого частого класса ("0") составляет 69,77%, что гораздо ниже полученных значений метрик качества моделей (accuracy). Поэтому модели можно считать вменяемыми.

Общий вывод: построил модели для классификации тарифов по пользователям. Наилучшие покзатели у Случайного леса. Удалось достичь, значения accuracy больше 0.75